
## ロジスティック回帰モデルとlightGBMモデルを組み合わせる．

### 背景
- ロジスティック回帰モデルが高精度だった
- 一般的により精度が高いとされるLightGBMモデルと組み合わせることで，より高い精度を出せる可能性がある．
### 使用するデータセット
- KDD99 10%
### 手法
- ロジスティック回帰モデルを二段階に分けて使用する．
- 1段階目：異常と正常の2値問題として分類する．
- 2段階目：異常と判断されたデータのみを抽出して，4つの異常クラスに分類する．
### 結果
- 学習時のロジスティック回帰モデルのパラメータによっては，高い精度で分類が可能．
- u2rの適合率は最大で42％程度
- 1段階目の分類ですでに`u2r`の多くが，`normal`に分類されてしまっている．
### 考察

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from utils_kdd99 import *

print_version()

python:      3.10.11
sklearn:     1.2.2
tensorflow:  2.12.0
keras:       2.12.0
numpy:       1.23.5
pandas:      1.5.3


In [2]:
# load data
x_train: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_train_df.pkl")
x_test: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_test_df.pkl")
y_train: pd.Series = pd.read_pickle("models/kdd99_features/y_train_df.pkl")
y_test: pd.Series = pd.read_pickle("models/kdd99_features/y_test_df.pkl")


In [3]:
# 第一段階　2値分類
def classification_normal_and_anomaly(X: pd.DataFrame, model: LogisticRegression | lgb.Booster)-> pd.Series:
    y_pred = model.predict(X)
    y_pred = pd.Series(y_pred, index=X.index,)
    return y_pred


In [4]:
# 第二段階　異常データの分類
def classification_anomalies(X: pd.DataFrame, model: LogisticRegression | lgb.Booster)-> pd.Series:
    y_pred = model.predict(X)
    # テストデータの予測 (予測クラス(0 or 1 or...)を返す)
    # y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに
    y_pred = pd.Series(y_pred, index=X.index)
    return y_pred

In [5]:
lr_binary_dir = 'models/logistic_regression_binary/'
lr_anomaly_dir = 'models/logistic_regression_anomaly/'
lgb_binary_dir = 'models/lightgbm/'
lr_binary_paths = os.listdir(lr_binary_dir)
lgb_binary_paths = os.listdir(lgb_binary_dir)


In [6]:
#
with open(lr_binary_dir + 'kdd99_38&penalty=l1&solver=liblinear&C=0.1.pkl', 'rb') as fp:
    model_lrb: LogisticRegression = pickle.load(fp)
y_pred_binary = classification_normal_and_anomaly(x_test, model_lrb)
predicted_indexes = y_pred_binary[y_pred_binary == 1].index
x_test_anomalies = x_test.loc[predicted_indexes]
y_test_anomalies = y_test.loc[predicted_indexes]
with open(lr_anomaly_dir + "kdd99_38&penalty=l1&solver=liblinear&C=0.1.pkl", 'rb') as fp:
    model_lra: LogisticRegression = pickle.load(fp)
y_pred_anomalies: pd.Series = classification_anomalies(x_test_anomalies, model_lra)

In [7]:
y_pred_normal = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
y_pred = pd.concat([y_pred_normal, y_pred_anomalies])

In [8]:
x_test.shape

(163027, 38)

In [9]:
y_test_binary = y_test.apply(lambda x: 0 if x == 1 else 1)
print(classification_report(y_test_binary, y_pred_binary))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99     32102
           1       1.00      1.00      1.00    130925

    accuracy                           1.00    163027
   macro avg       0.99      1.00      0.99    163027
weighted avg       1.00      1.00      1.00    163027



In [10]:
print(classification_report(y_test.sort_index(), y_pred.sort_index(), target_names=correspondences.keys()))


              precision    recall  f1-score   support

         dos       1.00      1.00      1.00    129181
      normal       0.99      0.99      0.99     32102
       probe       0.98      0.85      0.91      1355
         r2l       0.84      0.75      0.79       372
         u2r       0.78      0.41      0.54        17

    accuracy                           1.00    163027
   macro avg       0.92      0.80      0.85    163027
weighted avg       1.00      1.00      1.00    163027



In [11]:
cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])
cm_1st

,normal,anomaly
true_normal,31890,212
true_anomaly,359,130566


In [12]:
cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())
cm_2nd

,dos,normal,probe,r2l,u2r
true_dos,129115,64,0,2,0
true_normal,135,31890,24,51,2
true_probe,4,198,1152,1,0
true_r2l,4,87,2,279,0
true_u2r,0,10,0,0,7
